# Miniproyecto 3 / ModSim

    - Rodrigo Zea
    - Sebastian Arriola

In [1]:
import numpy as np
import math

## Generadores de Números Random (falta implementarlos)

In [3]:
def random_time(s, lam=5):
    """Generates random time after t, following nonhomogeneous Poisson process.
    """
    return s + np.random.poisson(lam=lam)

## Simulación para N Servidores

In [4]:
# Simulation Variables and Initialization
SERVERS = 5
# Offline Time
T = 100
# arrivals by time t
na = 0
# number of customers served by servers
customers_attended_per_server = [0 for _ in range(SERVERS)]
# System State
n = 0
current_server_state = [0 for _ in range(SERVERS)]
# for output data, keep track of arrival times and departures of customers
arrivals, departures = {}, {}
# time past T when last customer departs
tp = 0
# initial time of next customer departure
next_departures = [math.inf for _ in range(SERVERS)]
# initial time of next customer arrival
ta = np.random.poisson()
# current time is set to next arrival
t = ta

while t <= T or n > 0:
    # attend a customer arrival
    if (ta <= np.amin(next_departures)):
        # advance simulation time to next arrival
        t = ta
        print(f'Attending customer arrival at time {t}...')
        # number of customers arrived by time t increases by 1
        na = na + 1
        # generate time of next customer arrival
        ta = random_time(t)
        # store arrival time of customer
        arrivals[na] = t
        # increase number of customer in system at time t
        n = n + 1
            
        # there is some server available, find next available server and queue
        if n < SERVERS:
            for i in range(SERVERS):
                if current_server_state[i] == 0:
                    current_server_state[i] = na
                    next_departures[i] = random_time(t)
                    print(f'Server {i} is empty, customer {na} was enqueued and will depart at {next_departures[i]}')
                    break
        
        # system is full
        else:
            print('Servers are full, queuing customer...')
        
        # DEBUG
        print(f'Number of customers by time {t} is {na}')
        print(f'Next arrival is at time {ta}')
        print('****************************\n')
    
    # attend a customer departure
    else:
        # get the index of the next departure
        min_index = math.inf
        for i in range(SERVERS):
            if next_departures[i] < min_index:
                min_index = i
        
        # advance simulation time to next departure
        t = next_departures[min_index]
        # increase amount of customers served by server n
        temp = customers_attended_per_server[min_index]
        customers_attended_per_server[min_index] = temp + 1
        # store output data
        departures[current_server_state[min_index]] = t
        
        # DEBUG
        print(f'Attending customer departure at time {t}...')
        print(f'Customer {current_server_state[min_index]} has left server {min_index} at time {t}')
        print(f'There are {n - 1} customers in the system.')

        # reset system state
        n = n - 1
        current_server_state[min_index] = 0
        next_departures[min_index] = math.inf

        # if there is a customer on the queue
        if n > SERVERS:
            m = np.amax(current_server_state)
            current_server_state[min_index] = m + 1
            next_departures[min_index] = random_time(t)
        print('************************************\n')

print(f'Total customers served: {np.sum(customers_attended_per_server)}')
for i in range(SERVERS):
    print(f'Server {i} served {customers_attended_per_server[i]}')

Attending customer arrival at time 3...
Server 0 is empty, customer 1 was enqueued and will depart at 9
Number of customers by time 3 is 1
Next arrival is at time 5
****************************

Attending customer arrival at time 5...
Server 1 is empty, customer 2 was enqueued and will depart at 13
Number of customers by time 5 is 2
Next arrival is at time 9
****************************

Attending customer arrival at time 9...
Server 2 is empty, customer 3 was enqueued and will depart at 12
Number of customers by time 9 is 3
Next arrival is at time 14
****************************

Attending customer departure at time 9...
Customer 1 has left server 0 at time 9
There are 2 customers in the system.
************************************

Attending customer departure at time 13...
Customer 2 has left server 1 at time 13
There are 1 customers in the system.
************************************

Attending customer departure at time 12...
Customer 3 has left server 2 at time 12
There are 0 cus